# 推荐系统中的 ItemCF（基于物品的协同过滤）
## 1. ItemCF基本概念
`ItemCF`（Item-Based Collaborative Filtering）是`基于物品的协同过滤算法`，其核心思想是：`如果用户喜欢物品A，那么他很可能也喜欢与A相似的物品`。

- 与`UserCF`的区别
    - `UserCF`：找到相似用户，推荐相似用户喜欢的物品
    - `ItemCF`：找到相似物品，推荐与用户历史喜好相似的物品

---
## 2. ItemCF算法原理
- 核心步骤: \
1.计算物品相似度 \
2.根据用户历史行为和物品相似度生成推荐

---
### 标准实现流程（高层）

1.准备数据（显式评分或隐式交互），构建 𝑅（user × item）。

2.计算 item-item 相似度矩阵 `sim`（或只保留每个 item 的 top-K 相似物品）。

3.对每个用户，基于其已交互物品和相似度矩阵计算候选物品分数并排序（过滤已交互）。

4.评估（Recall@K / NDCG@K / Precision@K 等），在验证集中调参（topK、min_support、shrinkage、相似度类型等）。

---
## 3. ItemCF实现细节
基础Python实现

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

class ItemCF:
    def __init__(self, k=20, n_recommend=10):
        self.k = k  # 相似物品数
        self.n_recommend = n_recommend  # 推荐物品数
        self.user_item_matrix = None
        self.item_similarity = None
        self.item_popularity = None
        
    def fit(self, user_item_data):
        """训练模型"""
        # 构建用户-物品矩阵
        self._build_user_item_matrix(user_item_data)
        # 计算物品相似度
        self._calculate_item_similarity()
        # 计算物品流行度
        self._calculate_item_popularity()
        
    def _build_user_item_matrix(self, data):
        """构建用户-物品矩阵"""
        users = sorted(set(data['user_id']))
        items = sorted(set(data['item_id']))
        
        self.user_ids = {user: idx for idx, user in enumerate(users)}
        self.item_ids = {item: idx for idx, item in enumerate(items)}
        self.id_to_item = {idx: item for item, idx in self.item_ids.items()}
        
        # 创建矩阵（可以使用稀疏矩阵优化）
        self.user_item_matrix = np.zeros((len(users), len(items)))
        
        for _, row in data.iterrows():
            user_idx = self.user_ids[row['user_id']]
            item_idx = self.item_ids[row['item_id']]
            rating = row.get('rating', 1)  # 隐式反馈默认为1
            self.user_item_matrix[user_idx, item_idx] = rating
    
    def _calculate_item_similarity(self):
        """计算物品相似度矩阵"""
        # 使用余弦相似度
        self.item_similarity = cosine_similarity(self.user_item_matrix.T)
        
        # 将对角线设为0（物品与自身的相似度排除）
        np.fill_diagonal(self.item_similarity, 0)
    
    def _calculate_item_popularity(self):
        """计算物品流行度"""
        self.item_popularity = np.sum(self.user_item_matrix > 0, axis=0)
    
    def recommend(self, user_id, filter_interacted=True):
        """为用户生成推荐"""
        if user_id not in self.user_ids:
            return self._recommend_for_cold_start()
            
        user_idx = self.user_ids[user_id]
        user_interactions = self.user_item_matrix[user_idx]
        
        # 获取用户交互过的物品
        interacted_items = np.where(user_interactions > 0)[0]
        
        # 计算预测评分
        scores = np.zeros(self.user_item_matrix.shape[1])
        
        for item_idx in interacted_items:
            # 获取当前物品的最相似物品
            similar_items = np.argsort(self.item_similarity[item_idx])[::-1][:self.k]
            
            for sim_item_idx in similar_items:
                if filter_interacted and sim_item_idx in interacted_items:
                    continue
                    
                similarity = self.item_similarity[item_idx, sim_item_idx]
                rating = user_interactions[item_idx]
                scores[sim_item_idx] += similarity * rating
        
        # 归一化
        for i in range(len(scores)):
            if scores[i] > 0:
                sim_sum = 0
                for item_idx in interacted_items:
                    sim_sum += abs(self.item_similarity[item_idx, i])
                if sim_sum > 0:
                    scores[i] /= sim_sum
        
        # 获取推荐物品
        recommended_items = np.argsort(scores)[::-1][:self.n_recommend]
        
        recommendations = []
        for item_idx in recommended_items:
            if scores[item_idx] > 0:
                recommendations.append({
                    'item_id': self.id_to_item[item_idx],
                    'score': scores[item_idx],
                    'popularity': self.item_popularity[item_idx]
                })
        
        return recommendations
    
    def _recommend_for_cold_start(self):
        """冷启动用户推荐（返回热门物品）"""
        popular_items = np.argsort(self.item_popularity)[::-1][:self.n_recommend]
        return [{'item_id': self.id_to_item[idx], 'score': 0, 'popularity': self.item_popularity[idx]} 
                for idx in popular_items]

## 4. 实际应用示例
A. 显式评分 —— 调整余弦 ItemCF（适合评分数据）

In [ ]:
# 依赖：numpy、pandas
import numpy as np
import pandas as pd

def build_user_item_matrix(ratings_df, user_col='user', item_col='item', rating_col='rating'):
    """
    返回：
      R (np.array) shape=(n_users, n_items) 填充 0 表示未评分
      user_idx, item_idx 字典映射 id -> 索引
      users, items 列表
    """
    users = ratings_df[user_col].unique()
    items = ratings_df[item_col].unique()
    user_idx = {u:i for i,u in enumerate(users)}
    item_idx = {j:i for i,j in enumerate(items)}
    R = np.zeros((len(users), len(items)), dtype=float)
    for _, row in ratings_df.iterrows():
        R[user_idx[row[user_col]], item_idx[row[item_col]]] = row[rating_col]
    return R, user_idx, item_idx, users, items

def itemcf_adjusted_cosine_fit(R, eps=1e-9):
    """
    R: numpy array shape (n_users, n_items), 0 表示未评分
    返回：similarity 矩阵 shape (n_items, n_items)
    """
    mask = (R != 0).astype(float)  # 标记已评分
    # 用户均值（只对已评分求平均）
    user_sum = R.sum(axis=1)
    user_count = mask.sum(axis=1)
    user_mean = np.divide(user_sum, user_count, out=np.zeros_like(user_sum), where=user_count!=0)
    # 去均值（仅对已评分位置）
    R_centered = R - (user_mean[:, None] * mask)
    # 计算协方差 / 余弦分子
    numerator = R_centered.T.dot(R_centered)   # shape (n_items, n_items)
    # 分母（每个 item 的平方和）
    denom = np.sqrt(np.sum(R_centered**2, axis=0))  # length n_items
    denom_matrix = denom[:, None] * denom[None, :] + eps
    sim = numerator / denom_matrix
    # 对角置 0
    np.fill_diagonal(sim, 0.0)
    return sim

def recommend_itemcf_explicit(R, sim, user_index, top_k_neighbors=50, N=10):
    """
    为单个用户生成 Top-N 推荐（显式）
    - 只使用该用户已评分物品的 top_k_neighbors 相似度
    """
    rated = np.where(R[user_index] != 0)[0]
    if len(rated) == 0:
        return []  # 冷启动：没有历史
    # 计算分子：sim[:, rated] dot (r_u_rated - mean_u)
    user_mean = R[user_index, rated].mean()
    R_centered_u = R[user_index, rated] - user_mean
    # 只取每个 item 的 top_k_neighbors 避免所有列都参与，节省计算 (可选)
    # 这里简单使用全部 rated 项的相似度子矩阵
    numer = sim[:, rated].dot(R_centered_u)
    denom = np.abs(sim[:, rated]).sum(axis=1) + 1e-9
    preds = user_mean + numer / denom
    # 排除用户已评分的项
    preds[rated] = -np.inf
    top_items = np.argsort(preds)[-N:][::-1]
    return top_items  # 返回 item 索引列表


使用说明：\
1.先把评分表` ratings_df `转换为矩阵` R`， 

2.调用` itemcf_adjusted_cosine_fit(R) `得到` sim`，

3.然后对每个用户调用` recommend_itemcf_explicit `得到` Top-N`（`把 item `索引映射回` item id`）。

---
B. 隐式交互（binary / 点击 / 购买） —— 共现 + cosine（常见，较快）

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

def build_binary_matrix(interactions_df, user_col='user', item_col='item'):
    users = interactions_df[user_col].unique()
    items = interactions_df[item_col].unique()
    u_idx = {u:i for i,u in enumerate(users)}
    i_idx = {j:i for i,j in enumerate(items)}
    rows = interactions_df[user_col].map(u_idx).values
    cols = interactions_df[item_col].map(i_idx).values
    data = np.ones(len(rows), dtype=np.float32)
    R = csr_matrix((data, (rows, cols)), shape=(len(users), len(items)))
    return R, u_idx, i_idx, users, items

def itemcf_implicit_cosine(R_csr, topk=None, shrink=0, eps=1e-9):
    """
    R_csr: scipy csr_matrix shape (n_users, n_items), binary or counts
    返回：sim dense 或 sparse（若 topk 非 None 则返回稀疏形式）
    公式：sim = coocc / sqrt(pop_i * pop_j)
    shrink: 可选收缩项（减少低共现对相似度的影响）
    """
    # co-occurrence: item-item 共现矩阵 (n_items, n_items)
    coocc = (R_csr.T).dot(R_csr)  # sparse matrix
    # item popularity
    pop = np.array(R_csr.sum(axis=0)).ravel()  # length n_items
    denom = np.sqrt(np.outer(pop, pop)) + eps
    sim = coocc.toarray() / denom
    # shrinkage (简单示例)
    if shrink > 0:
        sim = sim * (coocc.toarray() / (coocc.toarray() + shrink))
    np.fill_diagonal(sim, 0.0)
    if topk is not None:
        # 只保留每行 topk，转为稀疏
        n_items = sim.shape[0]
        rows, cols, vals = [], [], []
        for i in range(n_items):
            idx = np.argpartition(-sim[i], topk)[:topk]
            for j in idx:
                if sim[i,j] > 0:
                    rows.append(i); cols.append(j); vals.append(sim[i,j])
        from scipy.sparse import csr_matrix
        sim_sparse = csr_matrix((vals, (rows, cols)), shape=sim.shape)
        return sim_sparse
    return sim  # dense


In [ ]:
推荐计算（隐式）：

# scores = R (n_users x n_items) dot sim (n_items x n_items) -> (n_users x n_items)
# 若 sim 是稀疏矩阵，使用稀疏乘法高效计算
scores = R_csr.dot(sim)  # 若 sim 稀疏 csr，需 sim 转 csc 或 csr 视情况
# 对每个用户把已交互的物品置 0，然后取 top-N

## 5.工程化 & 优化技巧（非常重要）

- 只保留` top-K `相似度：对每个` item `只存` top 50~200 `个相似` item`，可把相似度矩阵变得稀疏，内存和计算都大幅减少。

- Shrinkage / significance weighting：对于共现次数小的` pair`，给相似度打折（避免噪音）。例如用公式：sim *= coocc / (coocc + shrink)。

- 归一化/去偏：显式评分用` adjusted cosine `或` subtract user mean`；隐式可用` TF-IDF `风格处理 `item`（抑制超流行 item）。

- 增量更新：完整重算` item-item `非常慢；生产上常把` coocc `计数保存并只更新被影响的` item `对（用户交互通常只影响少数 item 的行/列）。

- 近邻搜索加速：对` item `向量（例如基于 SVD / embedding）用 ANN（Annoy、Faiss）做近邻检索，避免 𝑂(𝑚^2)的全对比。

- 稀疏矩阵乘法：用 scipy.sparse 做乘法（scores = R * sim）比 dense 快、内存友好。

- 时间衰减：对交互按时间加权（新交互权重大），防止历史行为主导推荐。

- 去热门物品惩罚：防止推荐总是热门` top-N`（增加新奇性、个性化）。

---
## 6.评价方式（离线实验）

- 划分方法：`按时间切分`（train=train interactions before t，test=后续交互），更贴近线上；或 leave-one-out（每用户保留一个交互为测试）。

- 指标：Recall@K / Precision@K / NDCG@K / MAP@K / HitRate@K / MRR。

- A/B：线下指标提升 ≠ 线上业务提升，需做在线实验验证（CTR、转化率、GMV 等）。

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

def evaluate_itemcf(model, test_data, top_k=10):
    """评估ItemCF模型性能"""
    precisions = []
    recalls = []
    
    for user_id in test_data['user_id'].unique():
        user_test_items = set(test_data[test_data['user_id'] == user_id]['item_id'])
        
        if len(user_test_items) == 0:
            continue
            
        # 生成推荐
        recommendations = model.recommend(user_id)
        recommended_items = set([rec['item_id'] for rec in recommendations[:top_k]])
        
        # 计算准确率和召回率
        if len(recommended_items) > 0:
            hit_items = user_test_items & recommended_items
            
            precision = len(hit_items) / len(recommended_items)
            recall = len(hit_items) / len(user_test_items)
            
            precisions.append(precision)
            recalls.append(recall)
    
    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    f1 = 2 * avg_precision * avg_recall / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
    
    return {
        'precision@k': avg_precision,
        'recall@k': avg_recall,
        'f1@k': f1
    }

# 使用示例
data = create_sample_data()
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

model = ItemCF(k=10, n_recommend=5)
model.fit(train_data)

metrics = evaluate_itemcf(model, test_data, top_k=3)
print("模型评估结果:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.3f}")

---
## 7.常见问题与对策

- 新用户冷启动：ItemCF 无历史则无法召回——用冷启动策略（热门榜、基于内容或引导问卷）。

- 新物品冷启动：ItemCF 需要物品有交互才能与其他物品建立联系——用物品属性做混合（content-based + ItemCF）。

- 长尾物品稀疏：可以合并同类目、用 item embedding 或增强数据（类别聚合）来缓解。

- 流行性偏差：可用 TF-IDF、逆文档频率或对相似度打折来平衡。

---
## 8.优缺点总结

- 优点：实现简单、解释性强（能直接说“因为你看过 X，所以推荐 Y”）、工程化容易（item-item 矩阵可离线计算和缓存）、对稀疏性较 UserCF 更稳健。

- 缺点：面对冷启动/新物品弱势；若只用共现容易产生热门泡沫；在极大规模物品集上计算仍有挑战（需稀疏化/近似）。

---
## 9. 进阶/变体（指路）

- Implicit ALS / Matrix Factorization：当用户或物品数量极大，因子分解通常能学到更低维的 item embeddings，可用 ANN 做召回或直接用因子相似度做 ItemCF。

- Session-based/Sequence-aware ItemCF：对 session 顺序敏感的场景（短会话推荐）可以结合序列模型（RNN、Transformer）或 session-based co-occurrence。

- Graph-based 方法：把用户和物品看作二分图，用图传播或随机游走（e.g., LightGCN）替代传统 ItemCF，效果通常更好但实现复杂。

---
## 10.小结 + 实践建议

- 起步：用**隐式 ItemCF（共现 + cosine）**快速上线一个 baseline（实现快，效果稳）。

- 若要提升效果：对显式评分采用 adjusted cosine；加 top-K、shrinkage、时间衰减与去流行性策略。

- 大规模：把相似度矩阵稀疏化并用近邻检索/embedding 做召回 + 再用排序模型（Learning-to-Rank）做最终排序。

---
## 应用示例 ：电影推荐系统

In [ ]:
import pandas as pd
import numpy as np

# 创建示例数据
def create_sample_data():
    """创建电影评分示例数据"""
    data = {
        'user_id': [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5],
        'item_id': [101, 102, 103, 104, 101, 102, 105, 103, 104, 106, 102, 105, 107, 101, 108],
        'rating': [5, 4, 3, 2, 4, 5, 3, 4, 3, 5, 4, 2, 3, 5, 4]
    }
    return pd.DataFrame(data)

# 使用ItemCF进行推荐
def movie_recommendation_example():
    # 准备数据
    data = create_sample_data()
    print("原始数据:")
    print(data)
    
    # 训练模型
    item_cf = ItemCF(k=3, n_recommend=5)
    item_cf.fit(data)
    
    # 为用户生成推荐
    for user_id in [1, 2, 3, 4, 5]:
        recommendations = item_cf.recommend(user_id)
        print(f"\n用户 {user_id} 的推荐:")
        for rec in recommendations:
            print(f"电影{rec['item_id']}: 评分{rec['score']:.3f}, 流行度{rec['popularity']}")
    
    # 分析物品相似度
    print("\n物品相似度矩阵:")
    similarity_df = pd.DataFrame(
        item_cf.item_similarity,
        index=item_cf.item_ids.keys(),
        columns=item_cf.item_ids.keys()
    )
    print(similarity_df.round(3))

# 运行示例
movie_recommendation_example()